In [1]:
# !pip install backtrader

In [2]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import backtrader as bt
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Custom Data Feed
class CustomPandasData(bt.feeds.PandasData):
    lines = ('signal',)
    params = (
        ('signal', -1),  # The index of the 'signal' column in the DataFrame
    )

In [4]:
import backtrader as bt

class IranStockCommission(bt.CommInfoBase):
    params = (
        ('commission', 0.00464),  # Commission rate for buy/sell
        ('stamp_duty', 0.0008),   # Stamp duty (applies only on sell)
        ('stocklike', True),      # Default to stock-like behavior
        ('commtype', bt.CommInfoBase.COMM_PERC),
    )

    def _getcommission(self, size, price, pseudoexec):
        """
        Returns the commission for a transaction.
        """
        commission = size * price * self.p.commission
        if size < 0:  # Selling
            commission += size * price * self.p.stamp_duty
        return commission

In [5]:
import backtrader as bt

# Define Buy and Hold Strategy
class BuyAndHold(bt.Strategy):
    def __init__(self):
        self.order = None  # To keep track of pending orders

    def start(self):
        self.data_len = len(self.data)
        print(f'Length of data: {self.data_len}')

    def next(self):
        # Check if this is the first bar and no position is open
        if len(self) == 1 and not self.position:
            available_cash = self.broker.get_cash()* 0.95
            print('available_cash', available_cash)
            size = int(available_cash / self.data.close[0])
            self.order = self.buy(size=size)
            print(size)
            print(self.data.close[0])
            print(f'Buy order placed at {self.data.close[0]} for size {size}')

        # Check if this is the last row and there is a position
        if len(self) == (self.data_len - 1) and self.position:
            self.order = self.sell(size=self.position.size)
            print(f'Last row: Sell order placed at {self.data.close[0]} for size {self.position.size}')

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                print(f'BUY EXECUTED, Price: {order.executed.price}, Cost: {order.executed.value}, Comm: {order.executed.comm}')
            elif order.issell():
                print(f'SELL EXECUTED, Price: {order.executed.price}, Cost: {order.executed.value}, Comm: {order.executed.comm}')
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            print(order, '===============')
            print(f'Order Canceled/Margin/Rejected at {self.data.datetime.datetime(0)}')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        print(f'Trade Profit, Gross: {trade.pnl}, Net: {trade.pnlcomm}')



In [6]:
# Define Strategy
class trade_based_on_labels(bt.Strategy):
    def __init__(self):
        self.signal = self.data.signal
        print(f'Initial Signal: {self.signal[0]}')
        self.order = None  # To keep track of pending orders

    def start(self):
        self.data_len = len(self.data)
        print(f'Length of data: {self.data_len}')

    def next(self):
        print(f'Next called at {self.data.datetime.datetime(0)} with signal {self.signal[0]}')
        
        if self.order:
            return  # If there's an order pending, do nothing

        if self.signal[0] == 1:
            if not self.position:
                available_cash = self.broker.get_cash()* 0.95
                size = int(available_cash / self.data.close[0])
                self.order = self.buy(size=size)
                print(f'Buy order placed at {self.data.close[0]} for size {size}')
        elif self.signal[0] == 0:
            if self.position:
                self.order = self.sell(size=self.position.size)
                print(f'Sell order placed at {self.data.close[0]} for size {self.position.size}')
        
        # Check if this is the last row
        if len(self) == self.data_len - 1 and self.position:
            self.order = self.sell(size=self.position.size)
            print(f'Last row: Sell order placed at {self.data.close[0]} for size {self.position.size}')

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            print(f'Order {order.getstatusname()} at {self.data.datetime.datetime(0)}')
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                print(f'BUY EXECUTED, Price: {order.executed.price}, Cost: {order.executed.value}, Comm: {order.executed.comm}')
            elif order.issell():
                print(f'SELL EXECUTED, Price: {order.executed.price}, Cost: {order.executed.value}, Comm: {order.executed.comm}')
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            print(f'Order Canceled/Margin/Rejected at {self.data.datetime.datetime(0)}')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        print(f'Trade Profit, Gross: {trade.pnl}, Net: {trade.pnlcomm}')

In [7]:
class trade_based_on_labels_sltp(bt.Strategy):
    params = (
        ('stop_loss', 0.05),  # Stop loss percentage (e.g., 2% below the buy price)
        ('take_profit', 0.1),  # Take profit percentage (e.g., 5% above the buy price)
    )
    
    def __init__(self):
        self.signal = self.data.signal
        self.order = None  # To keep track of pending orders
        self.buy_price = None  # To store the price at which the stock was bought

    def start(self):
        self.data_len = len(self.data)
        print(f'Length of data: {self.data_len}')

    def next(self):
        print(f'Next called at {self.data.datetime.datetime(0)} with signal {self.signal[0]}')
        
        if self.order:
            return  # If there's an order pending, do nothing

        if self.signal[0] == 1:
            if not self.position:
                available_cash = self.broker.get_cash()* 0.95
        if len(self) == self.data_len - 1 and self.position:
            self.order = self.sell(size=self.position.size)
            print(f'Last row: Sell order placed at {self.data.close[0]} for size {self.position.size}')

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            print(f'Order {order.getstatusname()} at {self.data.datetime.datetime(0)}')
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                print(f'BUY EXECUTED, Price: {order.executed.price}, Cost: {order.executed.value}, Comm: {order.executed.comm}')
            elif order.issell():
                print(f'SELL EXECUTED, Price: {order.executed.price}, Cost: {order.executed.value}, Comm: {order.executed.comm}')
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
           
            print(f'Order Canceled/Margin/Rejected at {self.data.datetime.datetime(0)}')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        print(f'Trade Profit, Gross: {trade.pnl}, Net: {trade.pnlcomm}')


In [21]:
# strategy_name = trade_based_on_labels_sltp
strategy_name = trade_based_on_labels
# strategy_name = BuyAndHold

strategy = 'trade_based_on_labels'
# strategy = 'trade_based_on_labels_sltp'
# strategy = 'BuyAndHold'
stock_name = 'وسکاب'
# data_path = './data/meta_labeling_کروی.csv.csv'
data_path ='./data/meta_labeling_'+stock_name+'.csv'
# data_path ='./data/backtest_pred_'+stock_name+'.csv'
data = pd.read_csv(data_path)
print(data)
# Convert the 'datetime' column to datetime type

data['datetime'] = pd.to_datetime(data['Date'])

# Initialize Cerebro engine
cerebro = bt.Cerebro()
# Convert the Pandas DataFrame to the custom data feed
data_feed = CustomPandasData(
    dataname=data,
    datetime=7,  # Specify the column index for 'datetime'
    high=2,
    low=3,
    open=1,
    close=4,
    volume=5,
)
cerebro.adddata(data_feed)

cerebro.broker.addcommissioninfo(IranStockCommission())
# Add the strategy to Cerebro
cerebro.addstrategy(strategy_name)

# Add Analyzers
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', timeframe=bt.TimeFrame.Days)
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

# Set the starting cash
cerebro.broker.setcash(10000000.0) #10 milion rial - 1 milion toman

# Run the strategy
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
results = cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Print analyzers
sharpe_ratio = results[0].analyzers.sharpe.get_analysis()
returns = results[0].analyzers.returns.get_analysis()
drawdown = results[0].analyzers.drawdown.get_analysis()

print(f"Sharpe Ratio: {sharpe_ratio['sharperatio']}")
print(f"Total Return: {returns['rtot']}")
print(f"Annual Return: {returns['rnorm']}")
print(f"Max Drawdown: {drawdown['max']['drawdown']}")

# Plot the result
cerebro.plot()
fig, ax = plt.subplots()
print(str(strategy_name))
cerebro.plot()[0][0].figure.savefig(fname = 'plot_'+stock_name+'_'+strategy)


          Date    Open    High     Low   Close     Volume  signal
0   2023-09-11  3722.0  3760.0  3714.0  3718.0  2159884.0       0
1   2023-09-12  3703.0  3790.0  3703.0  3752.0   789684.0       0
2   2023-09-13  3740.0  3779.0  3719.0  3759.0  1876156.0       0
3   2023-09-17  3799.0  3810.0  3719.0  3747.0   622000.0       0
4   2023-09-18  3797.0  3799.0  3720.0  3750.0  1725893.0       0
..         ...     ...     ...     ...     ...        ...     ...
60  2023-12-09  1914.0  1914.0  1884.0  1895.0  2387640.0       0
61  2023-12-10  1900.0  1920.0  1900.0  1914.0  3468184.0       0
62  2023-12-11  1912.0  1931.0  1900.0  1922.0  4789781.0       0
63  2023-12-12  1916.0  1950.0  1910.0  1915.0  5139490.0       0
64  2023-12-13  1938.0  1939.0  1908.0  1920.0  3071520.0       0

[65 rows x 7 columns]
Starting Portfolio Value: 10000000.00
Initial Signal: 0.0
Length of data: 0
Next called at 2023-09-11 00:00:00 with signal 0.0
Next called at 2023-09-12 00:00:00 with signal 0.0
Next ca

<IPython.core.display.Javascript object>

<class '__main__.trade_based_on_labels'>


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>